# Load needed modules

In [1]:
# modules:
import pandas as pd
import os
from openai import OpenAI
import numpy as np

import re



# get external variables or functions:
import src.API_key as key

# Load data to feed ChatGPT
> this should be the processed data by "EXCEL aufbereiten"

define the name of the data set you want to read (must be ".xlsx" file):

In [2]:
nameData_long = "class_LEB_CH 9.1_long"
nameData_wide = "class_LEB_CH 9.1"

In [3]:
print(os.getcwd())
# Get the current working directory
os.chdir("data") 
directory = os.getcwd()

c:\DATEN\PHD\Steffen_LEB\LEB generieren


In [4]:
# print the current working directory
print(directory)

c:\DATEN\PHD\Steffen_LEB\LEB generieren\data


In [5]:
# List files in the current working directory
files = os.listdir('.')
# Display the list of files
print(files)

['class_LEB_CH 9.1.xlsx', 'class_LEB_CH 9.1_long.xlsx']


In [6]:
# Load the .xlsx file into a DataFrame
df = pd.read_excel(directory + "/" + nameData_long + ".xlsx")

print(df.__len__())
print(df['Name'].unique())
#print(df['competences'])

38
['Albiez, Elias' 'Brennert, Marlon' 'Carbone, Valerio' 'Dorner, Sebastian'
 'Jung, Emily' 'Knab, Alicia' 'Knopf, Ben' 'Kraut, Lars' 'Kraut, Laura'
 'Kunzmann, Emanuel' 'Mresh, Joudi' 'Naimi, Yara' 'Neff, Florian'
 'Öztürk, Rana' 'Schwaß, Niklas' 'Stark, Amy' 'Still, Julian'
 'Thomas, Lovetta' 'Vera, Anthony']


c:\Users\fenn\AppData\Local\R-MINI~1\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# Set up ChatGPT

## 1. create prompt text

In [7]:
# Function to create the dynamic text
def create_prompt_learning_report(data):
    reports = []
    shown_names = set()
    for index, row in data.iterrows():
        name = row['Name']
        if name not in shown_names:
            shown_names.add(name)
            oral_mark_wording_inv = row['OralMark_wording_inv']
            oral_mark_wording_int = row['OralMark_wording_int']
            
            if pd.isna(oral_mark_wording_inv):
                reports.append(None)
            else:                
                condition = data['Name'] == name
                data_checks = data[condition]

                # Construct the text
                report = f"""Ich möchte einen Lernentwicklungsbericht für einen Schüler namens {name} schreiben. Verwende hierbei die dritte Person, um die Lernentwicklung des Schülers sachlich zu beschreiben. Achte hierbei auf das Geschlecht des Namens. Verwende nur den Vornamen des Schülers. Dieser Lernentwicklungsbericht sollte nur aus einem einzigen Paragraphen bestehen und folgend aufgebaut sein:"""
                report += f""" 1. Schreibe ein Satz zur mündlichen Note im Unterricht, ohne das Wort "mündlich" zu verwenden. 
                2. Schreibe zu jeder Kontrolle zwei bis drei Sätze, wobei das Niveau des Schülers am Anfang genannt wird. Beachte hierbei zu jedem Textblock der jeweiligen Kontrolle etwas zu schreiben, wobei "UND" für verschiedene Fähigkeiten innerhalb einer erworbenen Kompetenz steht. Schreibe pro Kontrolle / Check jedoch nicht mehr als 40 Worte. Verwende nicht das Wort "Kontrolle" oder "Check". 
                3. Am Schluss des Paragraphen soll "(Fe)" stehen."""
                report += f"""
                Hierbei übergebe ich dir im Folgenden, die dafür relevanten Informationen:     
                """
                report += f"""zu 1. mündliche Note:  
                gezeigtes Engagement im Unterricht: {oral_mark_wording_inv}, sowie gezeigtes Interesse im Unterricht: {oral_mark_wording_int}.
                """
                report += f"""zu 2. Kontrollen:"""
                
                for index, row in data_checks.iterrows():
                    check = row['Check']
                    competences = re.sub('\xad', '', row['competences'])
                    wording = row['wording']
                    report += f"""
                    {check}:
                    Niveau der  erworbene Kompetenzen: {wording}

                    Kompetenzen:
                    {competences}
                    
                    """
                    
                reports.append(report)
    return reports

promptText = create_prompt_learning_report(df)
print(promptText)

[None, None, 'Ich möchte einen Lernentwicklungsbericht für einen Schüler namens Carbone, Valerio schreiben. Verwende hierbei die dritte Person, um die Lernentwicklung des Schülers sachlich zu beschreiben. Achte hierbei auf das Geschlecht des Namens. Verwende nur den Vornamen des Schülers. Dieser Lernentwicklungsbericht sollte nur aus einem einzigen Paragraphen bestehen und folgend aufgebaut sein: 1. Schreibe ein Satz zur mündlichen Note im Unterricht, ohne das Wort "mündlich" zu verwenden. \n                2. Schreibe zu jeder Kontrolle zwei bis drei Sätze, wobei das Niveau des Schülers am Anfang genannt wird. Beachte hierbei zu jedem Textblock der jeweiligen Kontrolle etwas zu schreiben, wobei "UND" für verschiedene Fähigkeiten innerhalb einer erworbenen Kompetenz steht. Schreibe pro Kontrolle / Check jedoch nicht mehr als 40 Worte. Verwende nicht das Wort "Kontrolle" oder "Check". \n                3. Am Schluss des Paragraphen soll "(Fe)" stehen.\n                Hierbei übergebe i

## 2. apply ChatGPT

Function for API call

In [8]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback
def basic_API_call(
    openai_api_key,
    template,
    model_name="gpt-4",
    max_tokens=150,
):

    prompt = PromptTemplate(template=template)
    # top_p_value = float("0")
    seed = 123
    llm_chain = LLMChain(
        prompt=prompt,
        llm=ChatOpenAI(
            # llm=OpenAI(
            temperature=0.0,
            openai_api_key=openai_api_key,
            model_name=model_name,
            max_tokens=max_tokens,
            # model_kwargs={"top_p": top_p_value, "seed": seed},
            model_kwargs={"seed": seed},
        ),
        return_final_only=False,
    )
    
    with get_openai_callback() as cb:
        response = llm_chain.invoke(
            {}
        )
        print(cb)
        
    return response

In [9]:
# response = basic_API_call(key.openai_api_key, promptText[16], model_name="gpt-4", max_tokens=1000)
# response['text']
# response

add LEBs to existing pupil data.frame

In [10]:
# load data set of pupils
df_wide = pd.read_excel(directory + "/" + nameData_wide + ".xlsx")
# Add an empty column "LEB_neu" to the DataFrame
df_wide['LEB_neu'] = None

c:\Users\fenn\AppData\Local\R-MINI~1\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
response_full = []

for i in range(0, len(promptText)):
    if promptText[i] is not None:
        # print(i)
        # print(promptText[i])
        response = basic_API_call(key.openai_api_key, promptText[i], model_name="gpt-4", max_tokens=1000)
            
        df_wide['LEB_neu'][i] = response['text']
        response_full.append(response)

c:\Users\fenn\AppData\Local\R-MINI~1\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Tokens Used: 965
	Prompt Tokens: 674
	Completion Tokens: 291
Successful Requests: 1
Total Cost (USD): $0.037680000000000005


C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

Tokens Used: 1242
	Prompt Tokens: 880
	Completion Tokens: 362
Successful Requests: 1
Total Cost (USD): $0.048119999999999996


C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

...

C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

Tokens Used: 1212
	Prompt Tokens: 877
	Completion Tokens: 335
Successful Requests: 1
Total Cost (USD): $0.04641


C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

Tokens Used: 1058
	Prompt Tokens: 672
	Completion Tokens: 386
Successful Requests: 1
Total Cost (USD): $0.04332


C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_wide['LEB_neu'][i] = response['text']
C:\Users\fenn\AppData\Local\Temp\ipykernel_341232\2683912681.py:9: SettingWithCopyWarni

In [12]:
print(os.getcwd())
os.chdir("../output") 
print(os.getcwd())
# set the current working directory
directory = os.getcwd()
# Save the DataFrame to an Excel file
df_wide.to_excel(directory + "/" + nameData_wide + "_LEB" + ".xlsx", index=False)


# Save the DataFrame to an Excel file
df_json = pd.DataFrame(response_full)
df_json.to_excel(directory + "/" + nameData_wide + "_fullResponse" + ".xlsx", index=False)

c:\DATEN\PHD\Steffen_LEB\LEB generieren\data
c:\DATEN\PHD\Steffen_LEB\LEB generieren\output
